In [ ]:
#install pymysql and pandas on Python
pip install pymysql
pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import pymysql

In [ ]:
#Group 5 connect to mysql
connection=pymysql.connect(host='127.0.0.1',
                           user='root',
                           password='Password123',
                           db='project',
                           cursorclass=pymysql.cursors.DictCursor)
print(connection)
cursor=connection.cursor()

In [ ]:
#Drop the exists tables
cursor.execute("Drop table if exists Fund_dim;")
cursor.execute("Drop table if exists Risk_dim;")
cursor.execute("Drop table if exists Returns_dim;")
cursor.execute("Drop table if exists Yield_dim;")
cursor.execute("Drop table if exists Expense_dim;")
cursor.execute("Drop table if exists Fund_fact;")
cursor.execute("Drop table if exists Firm_fact;")
cursor.execute("Drop table if exists Firm_dim;")
cursor.execute("Drop table if exists PE_dim;")
cursor.execute("Drop table if exists EPS_dim;")

In [ ]:
#Read csv file
data1 = pd.read_csv ('ETFs.csv',parse_dates=['inception_date'])
data2 = pd.read_csv ('companies.csv')
data3 = pd.read_csv ('full_financial_stocks_raw.csv')

In [ ]:
#Transfering file into dataframe with need columns
df1=pd.DataFrame(data1,columns=['fund_symbol','fund_extended_name','investment_type','size_type'])
df2=pd.DataFrame(data1,columns=['fund_extended_name','fund_treynor_ratio_3years','fund_treynor_ratio_5years','fund_treynor_ratio_10years'])
df3=pd.DataFrame(data1,columns=['fund_extended_name','fund_return_3months','fund_return_1year','fund_return_3years','fund_return_5years'])
df4=pd.DataFrame(data1,columns=['fund_extended_name','fund_yield'])
df5=pd.DataFrame(data1,columns=['fund_extended_name','net_asset_value','fund_net_annual_expense_ratio'])
df6=pd.DataFrame(data1,columns=['top10_holdings'])
df7=pd.DataFrame(data2,columns=['ticker','company name','short name','exchange','market cap'])
df8=pd.DataFrame(data3,columns=['symbol','security_name','regularMarketPrice'])
df9=pd.DataFrame(data3,columns=['symbol','security_name','forwardEps'])
df10=pd.DataFrame(data3,columns=['symbol','security_name','forwardPE'])
df12=pd.DataFrame(data1,columns=['fund_extended_name','top10_holdings','inception_date'])

In [ ]:
#Split the hodings to different columns
df6['top10_holdings'] =(df6['top10_holdings'].astype(str)) #convert data type to string
df6=df6.top10_holdings.str.split(',',expand=True) #split holding column
df6 = pd.concat([df12, df6], axis=1) #concat two column
df6.columns = ['fund_extended_name','top10_holdings','inception_date','holding1','holding2','holding3','holding4','holding5','holding6','holding7','holding8','holding9','holding10','holding11','holding12','holding13','holding14','holding15','holding16']

#Change NaN to None which allows mysql read
df1=df1.replace({np.nan:None})
df2=df2.replace({np.nan:None})
df3=df3.replace({np.nan:None})
df4=df4.replace({np.nan:None})
df5=df5.replace({np.nan:None})
df6=df6.replace({np.nan:None})
df7=df7.replace({np.nan:None})
df8=df8.replace({np.nan:None})
df9=df9.replace({np.nan:None})
df10=df10.replace({np.nan:None})

In [ ]:
#Convert dataframe to list
dfa=df1.values.tolist()
dfb=df2.values.tolist()
dfc=df3.values.tolist()
dfd=df4.values.tolist()
dfe=df5.values.tolist()
dff=df6.values.tolist()
dfg=df7.values.tolist()
dfh=df8.values.tolist()
dfi=df9.values.tolist()
dfj=df10.values.tolist()

In [ ]:
#Create fund table in Mysql
sql1="""Create Table if not exists Fund_dim (
fund_symbol VARCHAR(255),
fund_extended_name VARCHAR(255),
investment_type VARCHAR(255),
size_type VARCHAR(255));"""
cursor=connection.cursor()
cursor.execute(sql1)
connection.commit()

#Create risk table in Mysql
sql2="""Create Table if not exists Risk_dim (
fund_extended_name VARCHAR(255),
fund_treynor_ratio_3years FLOAT,
fund_treynor_ratio_5years FLOAT,
fund_treynor_ratio_10years FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql2)
connection.commit()

#Create returns table in Mysql
sql3="""Create Table if not exists Returns_dim (
fund_extended_name VARCHAR(255),
fund_return_3months FLOAT,
fund_return_1year FLOAT,
fund_return_3years FLOAT,
fund_return_5years FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql3)
connection.commit()

#Create yield table in Mysql
sql4="""Create Table if not exists Yield_dim (
fund_extended_name VARCHAR(255),
fund_yield FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql4)
connection.commit()

#Create expense table in Mysql
sql5="""Create Table if not exists Expense_dim (
fund_extended_name VARCHAR(255),
net_asset_value FLOAT,
fund_net_annual_expense_ratio FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql5)
connection.commit()

#Create fund table in Mysql
sql6="""Create Table if not exists Fund_fact (
fund_extended_name VARCHAR(255),
top10_holdings VARCHAR(1000),
inception_date DATE,
holding1 VARCHAR(255),
holding2 VARCHAR(255),
holding3 VARCHAR(255),
holding4 VARCHAR(255),
holding5 VARCHAR(255),
holding6 VARCHAR(255),
holding7 VARCHAR(255),
holding8 VARCHAR(255),
holding9 VARCHAR(255),
holding10 VARCHAR(255),
holding11 VARCHAR(255),
holding12 VARCHAR(255),
holding13 VARCHAR(255),
holding14 VARCHAR(255),
holding15 VARCHAR(255),
holding16 VARCHAR(255));"""
cursor.execute(sql6)
connection.commit()

#Create firm table in Mysql
sql7="""Create Table if not exists Firm_dim (
ticker VARCHAR(255),
company_name VARCHAR(255),
short_name VARCHAR(255),
exchange VARCHAR(255),
market_cap FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql7)
connection.commit()

#Create firm fact table in Mysql
sql8="""Create Table if not exists Firm_fact (
symbol VARCHAR(255),
security_name VARCHAR(255),
regularMarketPrice FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql8)
connection.commit()

#Create EPS table in Mysql
sql9="""Create Table if not exists EPS_dim (
symbol VARCHAR(255),
security_name VARCHAR(255),
forwardEps FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql9)
connection.commit()

#Create PE table in Mysql
sql10="""Create Table if not exists PE_dim (
symbol VARCHAR(255),
security_name VARCHAR(255),
forwardPE FLOAT);"""
cursor=connection.cursor()
cursor.execute(sql10)
connection.commit()

In [ ]:
#Insert list to Mysql
cursor.executemany("INSERT INTO Fund_dim VALUES (%s,%s,%s,%s);",dfa)
connection.commit()
cursor.executemany("INSERT INTO Risk_dim VALUES (%s,%s,%s,%s);",dfb)
connection.commit()
cursor.executemany("INSERT INTO Returns_dim VALUES (%s,%s,%s,%s,%s);",dfc)
connection.commit()
cursor.executemany("INSERT INTO Yield_dim VALUES (%s,%s);",dfd)
connection.commit()
cursor.executemany("INSERT INTO Expense_dim VALUES (%s,%s,%s);",dfe)
connection.commit()
cursor.executemany("INSERT INTO Fund_fact VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);",dff)
connection.commit()
cursor.executemany("INSERT INTO Firm_dim VALUES (%s,%s,%s,%s,%s);",dfg)
connection.commit()
cursor.executemany("INSERT INTO Firm_fact VALUES (%s,%s,%s);",dfh)
connection.commit()
cursor.executemany("INSERT INTO EPS_dim VALUES (%s,%s,%s);",dfi)
connection.commit()
cursor.executemany("INSERT INTO PE_dim VALUES (%s,%s,%s);",dfj)
connection.commit()

NameError: ignored

In [ ]:
#Close connection
connection.close()